In [1]:
# import dependencies
import pandas as pd
import numerapi
import sklearn.linear_model
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor,  Pool
from tqdm import tqdm
import lightgbm
import numpy as np
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import math
from skimage.transform import resize
from skimage.io import imsave
from skimage.io import imread

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# download the latest training dataset (takes around 30s)
# training_data = pd.read_csv("latest_numerai_training_data.csv.xz")
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,...,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.50
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,...,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,...,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.25
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,...,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.25
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,...,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [3]:
# download the latest tournament dataset (takes around 30s)
# tournament_data = pd.read_csv("latest_numerai_tournament_data.csv.xz")
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
tournament_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,...,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
0,n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,...,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
1,n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,...,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
2,n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,...,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
3,n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,...,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
4,n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,...,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [13]:
feature_cols = tournament_data.columns[tournament_data.columns.str.startswith('feature')]

In [34]:
TARGET_NAME="target"

In [15]:
from tensorflow.keras.models import Model


def autoencode_model(input_size, output_size, latendim):
    
    # print("AA", input_size)
    input_data = layers.Input(shape=(input_size,))
    # x = layers.BatchNormalization()(input_data)
    # x = layers.GaussianNoise(1)(x)
    x = layers.Dense(units=512*4, activation="relu")(input_data)
    # x = layers.BatchNormalization()(x)
    # x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(units=512*2, activation="relu")(x)
    #x = layers.BatchNormalization()(x)
    #x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(units=512, activation="relu")(x)
    #x = layers.BatchNormalization()(x)
    #x = layers.Dropout(rate=0.5)(x)
    decode = layers.Dense(units=latendim, activation="sigmoid")(x)
    # x = layers.GaussianNoise(1)(x)
    # x = layers.BatchNormalization()(decode)

    # x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(units=512, activation="relu")(decode)
    # x = layers.BatchNormalization()(x)
    # x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(units=512*2, activation="relu")(x)
    # x = layers.BatchNormalization()(x)
    # x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(units=512*4, activation="relu")(x)
    # x = layers.BatchNormalization()(x)
    # x = layers.Dropout(rate=0.5)(x)

    output = layers.Dense(units=output_size)(x)
    
    return input_data, decode, output

In [21]:
int_input, int_decode, int_output = autoencode_model(len(feature_cols), len(feature_cols), 50)

In [22]:
model = Model(int_input, int_output)

    
optimizer=keras.optimizers.Adam()
model.compile(optimizer=optimizer,
                         loss='mse')

model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 310)]             0         
_________________________________________________________________
dense_16 (Dense)             (None, 2048)              636928    
_________________________________________________________________
dense_17 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_18 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_19 (Dense)             (None, 50)                25650     
_________________________________________________________________
dense_20 (Dense)             (None, 512)               26112     
_________________________________________________________________
dense_21 (Dense)             (None, 1024)             

In [23]:
allData = pd.concat((training_data[feature_cols], tournament_data[feature_cols])).astype(np.float16)

In [24]:
allData

,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,...,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46
0,0.00,0.50,0.25,0.00,0.50,0.25,0.25,0.25,0.75,0.75,...,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75
1,0.00,0.00,0.00,0.25,0.50,0.00,0.00,0.25,0.50,0.50,...,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00
2,0.25,0.50,0.25,0.25,1.00,0.75,0.75,0.25,0.00,0.25,...,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75
3,1.00,0.00,0.00,0.50,0.50,0.25,0.25,0.75,0.25,0.50,...,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00
4,0.25,0.25,0.25,0.25,0.00,0.25,0.50,0.25,0.25,0.50,...,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581157,0.50,0.25,0.25,0.25,0.00,0.00,0.00,0.00,0.25,0.50,...,0.00,0.00,0.00,0.50,0.50,0.00,0.25,0.50,0.25,0.25
1581158,0.00,0.50,0.50,0.00,1.00,1.00,1.00,0.00,0.75,0.75,...,0.25,0.00,0.00,0.75,0.25,0.00,0.50,0.50,0.75,0.25
1581159,0.00,0.50,0.50,0.00,0.50,1.00,1.00,0.00,0.75,0.50,...,0.25,0.00,0.50,0.00,0.00,0.25,0.50,0.00,0.25,0.50
1581160,0.00,0.75,0.50,0.00,0.25,0.25,0.25,0.00,1.00,1.00,...,0.00,0.00,0.25,0.25,0.25,0.00,0.00,0.50,0.25,0.25


In [25]:
model.fit(allData, allData, epochs=50, batch_size=2500)

Epoch 1/50
834/834 [==============================] - 24s 28ms/step - loss: 0.0744
Epoch 2/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0407
Epoch 3/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0322
Epoch 4/50
834/834 [==============================] - 24s 28ms/step - loss: 0.0288
Epoch 5/50
834/834 [==============================] - 24s 28ms/step - loss: 0.0276
Epoch 6/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0269
Epoch 7/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0263
Epoch 8/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0258
Epoch 9/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0254
Epoch 10/50
834/834 [==============================] - 24s 29ms/step - loss: 0.0251
Epoch 11/50
834/834 [==============================] - 25s 29ms/step - loss: 0.0247
Epoch 12/50
834/834 [==============================] - 25s 30ms/step - loss: 0.0244
E

In [26]:
encoder = Model(int_input, int_decode)

In [27]:
encoder.save_weights('compresed-data-kaz-10')

In [28]:
encoder.load_weights('compresed-data-kaz-10')

In [29]:
with tf.device('/cpu:0'):    
    allData = encoder.predict(allData, batch_size=500)

In [30]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

# StandardScaler MinMaxScaler
scaler = MinMaxScaler(copy=False)
scaler.fit_transform(allData)

array([[0.6251146 , 0.4461584 , 0.531525  , ..., 0.53044534, 0.4580845 ,
        0.5195337 ],
       [0.36768994, 0.60206205, 0.5003171 , ..., 0.673376  , 0.33686873,
        0.46793348],
       [0.67587686, 0.36062908, 0.37664944, ..., 0.6732209 , 0.18618156,
        0.43603677],
       ...,
       [0.45727473, 0.21015526, 0.24148655, ..., 0.56577975, 0.2942965 ,
        0.5229786 ],
       [0.34052116, 0.2758993 , 0.29269752, ..., 0.5407742 , 0.5332389 ,
        0.65642834],
       [0.35437393, 0.6194496 , 0.51645815, ..., 0.407801  , 0.39971817,
        0.5244068 ]], dtype=float32)

In [31]:

indx = len(training_data)
train_data =  allData[0:indx]

In [32]:
train_data = pd.DataFrame(train_data)

In [35]:
train_data['target'] = training_data[TARGET_NAME]
train_data['era'] = training_data['era']

In [36]:
train_data.to_csv('train_data-kaz-10.csv')

In [38]:
train_data = allData[indx:]
train_data = pd.DataFrame(train_data)
train_data['target'] = tournament_data[TARGET_NAME]
train_data['era'] = tournament_data['era']
train_data['type'] = tournament_data['data_type']
train_data['id'] = tournament_data['id']
train_data

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,target,era,type,id
0,0.684969,0.537315,0.644790,0.475414,0.522303,0.561284,0.441950,0.583328,0.357338,0.583919,...,0.652358,0.747293,0.560961,0.417274,0.602620,0.593732,0.25,era121,validation,n0003aa52cab36c2
1,0.462252,0.545916,0.394655,0.378866,0.569654,0.708049,0.495076,0.426700,0.565530,0.217472,...,0.594471,0.464464,0.739604,0.395773,0.587933,0.522045,0.50,era121,validation,n000920ed083903f
2,0.582050,0.557493,0.535555,0.323159,0.506384,0.605278,0.365147,0.181733,0.332626,0.691691,...,0.639979,0.540645,0.414977,0.393736,0.625929,0.519788,1.00,era121,validation,n0038e640522c4a6
3,0.419429,0.462896,0.448324,0.513063,0.390499,0.412967,0.390469,0.337310,0.549886,0.500408,...,0.414323,0.467434,0.382670,0.339632,0.606715,0.264071,0.50,era121,validation,n004ac94a87dc54b
4,0.632080,0.405627,0.395501,0.304732,0.428091,0.342243,0.480964,0.703205,0.334977,0.623257,...,0.541503,0.377432,0.390525,0.393758,0.299309,0.440200,0.75,era121,validation,n0052fe97ea0c05f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581157,0.518760,0.533550,0.415846,0.494752,0.739019,0.683554,0.731919,0.175976,0.367027,0.334295,...,0.426593,0.297173,0.257343,0.516781,0.273023,0.504643,NaN,eraX,live,nffa978d27d95877
1581158,0.515407,0.405475,0.435196,0.319632,0.411410,0.381248,0.457293,0.439325,0.519259,0.543545,...,0.199716,0.574608,0.347615,0.558244,0.406801,0.427814,NaN,eraX,live,nffc0ac8f014eee1
1581159,0.457275,0.210155,0.241487,0.438986,0.375662,0.461163,0.633173,0.674515,0.520559,0.394763,...,0.486562,0.418830,0.463560,0.565780,0.294297,0.522979,NaN,eraX,live,nffc3d2a31602573
1581160,0.340521,0.275899,0.292698,0.343519,0.309348,0.463239,0.433467,0.608854,0.602733,0.532677,...,0.290211,0.166307,0.144287,0.540774,0.533239,0.656428,NaN,eraX,live,nffcbbd1c5eb2f75


In [39]:
train_data.to_csv('tournament_data-kaz-10.csv')